In [1]:
import base58
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [2]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, testnet):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, testnet):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if testnet:
        prefix = b"\xc4"
    else:
        prefix = b"\x05"
    return encode_base58_checksum(prefix + rs_hash)

def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)

# Bitcoin core regtest tx tester
This notebook will start up bitcoind in regtest mode, create transactions using python scripts, and broadcast them to validate that they work. Four transactions are tested here:<br><br>

- tx1 initial tx to fund customer. 1 output 
    - Send 11.3 btc to a P2SH-P2WPKH address. This will be used to fund the customer.
- tx2 funding tx with change. 2 outputs
    - 10.2 btc to an escrow (2-of-2 multisig) 
    - 1 btc to a change address
- tx3 merch close. 1 output
    - 10.1 btc to p2wsh with two ways to spend from it:
        - 2 of 2 multisig signed by cust_pk and merch_pk (cust-close-tx)
        - to_self_delay to merch_close_pk (merch-claim-tx)

Funding private key is used to fund the escrow tx. <br> 
Merch private key and Cust private key are used to generate the pubkeys in the escrow tx. <br>
The Cust private key is also used for the P2WPKH change output.

In [3]:
funding_privkey_hex = "1111111111111111111111111111111111111111111111111111111111111111"
funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(funding_privkey_hex)).hex()

merch_privkey_hex = "2111111111111111111111111111111111111111111111111111111111111111"
merch_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_privkey_hex)).hex()

cust_privkey_hex = "3111111111111111111111111111111111111111111111111111111111111111"
cust_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_privkey_hex)).hex()

change_privkey_hex = "4111111111111111111111111111111111111111111111111111111111111111"
change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(change_privkey_hex)).hex()

merch_close_privkey_hex = "5111111111111111111111111111111111111111111111111111111111111111"
merch_close_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_close_privkey_hex)).hex()

Start up regtest mode, delete any history so we are starting from scratch.
Mine 101 blocks

In [52]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon -minrelaytxfee=0 -fallbackfee=0.0002")
time.sleep(1.5)

# generate 101 blocks so we can fund transactions
address = subprocess.getoutput("bitcoin-cli -regtest getnewaddress")
result = subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 101 {addr}".format(addr=address))

blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

print("blockcount: " + str(blockcount))

blockcount: 101


Generate base58 address for the escrow funder

In [53]:
# Generate p2sh-p2wpkh address to fund the escrow funder
privkey = bytes.fromhex(funding_privkey_hex)
public_key = privkey_to_pubkey(privkey)
p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("Address: " + p2sh_p2wpkh_address)

Address: 2NEp346GsvyPAu3X9oKGjrzLXc8vw8CqG7T


Send btc to the escrow funder

In [54]:
txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + p2sh_p2wpkh_address + " 11.3")
print(txid_1)

98870084fb073f9ad7f919f3fcb7e744a5f36294b80a9cd2ed5f6728c47f0203


In [55]:
# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

index: 1


In [56]:
subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 1 {addr}".format(addr=address));

Generate raw escrow funding transaction

In [57]:
raw_escrow_tx = subprocess.getoutput("python funding_tx_with_change.py" 
                        + " --funding_privkey " + funding_privkey_hex
                        + " --txid " + txid_1
                        + " --index " + str(index)
                        + " --amount_btc " + "11.3"
                        + " --merch_pubkey " + merch_pubkey_hex
                        + " --cust_pubkey " + cust_pubkey_hex
                        + " --change_pubkey " + change_pubkey_hex
                        + " --escrow_btc " + "10.2"
                        + " --change_btc " + "1")
print(raw_escrow_tx)

0200000000010103027fc428675fedd29c0ab89462f3a544e7b7fcf319f9d79a3f07fb840087980100000017160014fc7250a211deddc70ee5a2738de5f07817351cefffffffff02fff6cb3c000000002200201f3abc4d03a3b61defffe4d163f25fcd0edc96f352ed1324385dc1ce28debd0200e1f50500000000160014bb524cfc4313630bdd51106c1a1b1fb58a3cea3c02483045022100b7c36d6e573251ea4c9d3f68bb2ae0eedc18acefbda8f011164c3c9134764fbb022057a1af2d0e78fbb97315f249044640e47ffb0eb50a717f01de13435f0b6be9860121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa00000000


Broadcast escrow funding transaction

In [58]:
escrow_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_escrow_tx + " 10000000")
# "true" flag means we are okay with absurdly high tx fee
print(escrow_txid)

0b5af206f958acf95ae7251d8288023890ce706f83642ddeeb71139a08e8312b


In [59]:
raw_escrow_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + escrow_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_escrow_tx1)
# print(decoded)

In [60]:
subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 1 {addr}".format(addr=address));

Create transaction spending from the change output of the escrow transaction

In [61]:
raw_merch_close_tx = subprocess.getoutput("python merch_close.py"        
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex      
                + " --merch_close_pubkey " + merch_close_pubkey_hex   
                + " --to_self_delay " + "10" # number of blocks to delay to-self output       
                + " --txid " + escrow_txid
                + " --index " + "0"
                + " --amount_btc " + "10.2"
                + " --output_btc " + "10.1")
print(raw_merch_close_tx)

decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_merch_close_tx)
# print(decoded)

020000000001012b31e8089a1371ebde2d64836f70ce90380288821d25e75af9ac58f906f25a0b0000000000ffffffff018060333c000000002200200b18302d7455b9fe65fd8455f80635806ae4961835097025074071a28c83d2e70400483045022100a6cf65e2639328fc23d4c5ecd301f883a5aa2482e6dd95a10fba008eff83de5d022063cb1c967eba7e3094cf0a85f302b75caa026d3acae7c4496b812a16ad64f84701483045022100ed98b4b395d5cc693cc6b9d8f70137ff79d415c8beac933d8e7d3b35ae5518b40220201d8f4ded1c183d4874098dbe1f6904de0b791a8978af46696d3998962635cb01475221024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c52ae00000000


In [62]:
merch_close_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_close_tx + " 1000000")
print(merch_close_txid)

b974c23818e533b89f15a9e7d3882070faeaf319fd121ffdedd366751f8f0137


In [63]:
merch_close_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_close_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_close_tx)
print(decoded)

{
  "txid": "b974c23818e533b89f15a9e7d3882070faeaf319fd121ffdedd366751f8f0137",
  "hash": "8361438a62f326eee6d64d624402a189b262ebb00201dc426bd686086ca21277",
  "version": 2,
  "size": 316,
  "vsize": 150,
  "weight": 598,
  "locktime": 0,
  "vin": [
    {
      "txid": "0b5af206f958acf95ae7251d8288023890ce706f83642ddeeb71139a08e8312b",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "3045022100a6cf65e2639328fc23d4c5ecd301f883a5aa2482e6dd95a10fba008eff83de5d022063cb1c967eba7e3094cf0a85f302b75caa026d3acae7c4496b812a16ad64f84701",
        "3045022100ed98b4b395d5cc693cc6b9d8f70137ff79d415c8beac933d8e7d3b35ae5518b40220201d8f4ded1c183d4874098dbe1f6904de0b791a8978af46696d3998962635cb01",
        "5221024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c52ae"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "valu

In [64]:
raw_merch_claim_tx = subprocess.getoutput("python merch_claim.py"   
                + " --merch_pubkey " + merch_pubkey_hex   
                + " --cust_pubkey " + cust_pubkey_hex   
                + " --merch_close_privkey " + merch_close_privkey_hex   
                + " --output_pubkey " + "024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1"    
                + " --to_self_delay " + "10" # number of blocks to delay to-self output         
                + " --txid " + merch_close_txid
                + " --index " + "0"
                + " --amount_btc " + "10.1"
                + " --output_btc " + "10")
print(raw_merch_claim_tx)


decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_merch_claim_tx)
print(decoded)

0200000000010137018f1f7566d3edfd1f12fd19f3eafa702088d3e7a9159fb833e51838c274b90000000000100000000100ca9a3b00000000160014b7cfef435e3701fdedb7a11164ae44d561698bb90347304402205dafda8a27ec63dcedb1f40b8c1c0ae37ef266ab29d097a94286f449804ced0302205c2dc17e2ecc3ed956628aff22b9aade3b9f049c2ddfc67fbd4e268f78e00381010070635221024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c52ae6760b2752102ab573100532827bd0e44b4353e4eaa9c79afbc93f69454a4a44d9fea8c45b5afac6800000000
{
  "txid": "02ac0833df4ca9d24be5b56e7b2e27e9b74199badff2036c94a41741c49b461c",
  "hash": "98dff334073d4855e2557e4b499abff688eba08909048665174a78ba1fd4b944",
  "version": 2,
  "size": 271,
  "vsize": 130,
  "weight": 517,
  "locktime": 0,
  "vin": [
    {
      "txid": "b974c23818e533b89f15a9e7d3882070faeaf319fd121ffdedd366751f8f0137",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "304

In [65]:
subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 15 {addr}".format(addr=address));

Only 1486 blocks mined, one less than the delay_to_self of 1487 blocks

In [66]:
merch_claim_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_claim_tx + " 1000000")
print(merch_claim_txid)

error code: -26
error message:
non-BIP68-final


Generate 1 more block and try again

In [67]:
subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 1 {addr}".format(addr=address));

In [68]:
merch_claim_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_claim_tx + " 10000000")
print(merch_claim_txid)

02ac0833df4ca9d24be5b56e7b2e27e9b74199badff2036c94a41741c49b461c


In [34]:
merch_claim_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_claim_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_claim_tx)
print(decoded)

{
  "txid": "21f2b678cd29e3028e0edea4d9735eced6814541b8f8a68758c6681020f58c12",
  "hash": "ae2869bce5af45f792b6566b63ff636298204c907942e30145c34b538a6789d0",
  "version": 2,
  "size": 274,
  "vsize": 130,
  "weight": 520,
  "locktime": 0,
  "vin": [
    {
      "txid": "7a2650c1a7463ac6fb744f1de08f2b0cc8e1238ca7b2ea61fa7f41f6fd432ec1",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3045022100fe23c5eb3f4d1469f65af5645ea7b3fc15e7dd764bd1dbb2a2e02322acecbc4f0220299b677f343812e98e9fec0634e3f7537a5eb57143d2e4dcfa2cb806e288fb3801",
        "",
        "635221024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c52ae6702cf05b2752102ab573100532827bd0e44b4353e4eaa9c79afbc93f69454a4a44d9fea8c45b5afac68"
      ],
      "sequence": 1487
    }
  ],
  "vout": [
    {
      "value": 10.00000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 b7c

In [22]:
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 